In [ ]:
import tensorflow as tf
import numpy as np
import os

tf.logging.set_verbosity(tf.logging.INFO)

BATCH_SIZE = 120
def weight_variable(shape, w=0.1):
    initial = tf.truncated_normal(shape, stddev=w)
    return tf.Variable(initial)

def bias_variable(shape, w=0.1):
    initial = tf.constant(w, shape=shape)
    return tf.Variable(initial)

def predicter(input_layer):
      # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = weight_variable(shape=[5, 5, 3, 64])
        conv = tf.nn.conv2d(input_layer, kernel, [1, 1, 1, 1], padding='SAME')
        biases = bias_variable(shape=[64], w=0.0)
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)

  # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool1')
  # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm1')

  # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = weight_variable(shape=[5, 5, 64, 64])
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = bias_variable(shape=[64], w=0.1)
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)

  # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm2')
  # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                         strides=[1, 2, 2, 1], padding='SAME', name='pool2')

  # local3
    with tf.variable_scope('local3') as scope:
    # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(pool2, [BATCH_SIZE, -1])
        dim = reshape.get_shape()[1].value
        weights = weight_variable(shape=[dim, 384])
        print('dim', weights.shape)
        biases = bias_variable(shape=[384],w=0.1)
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)

  # local4
    with tf.variable_scope('local4') as scope:
        weights = weight_variable(shape=[384, 192])
        biases = bias_variable(shape=[192])
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)

  # linear layer(WX + b),
  # We don't apply softmax here because
  # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
  # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        weights = weight_variable(shape=[192, 10])
        biases = bias_variable(shape=[10])
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)

    return softmax_linear

def cnn_model_fn(features, labels, mode):
#   """Model function for CNN."""
  
    # Input Layer
    # input layer shape should be [batch_size, image_width, image_height, channels] for conv2d
    # set batch_size = -1 means batch_size = the number of input
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])

#     # Convolutional Layer #1 and Pooling Layer #1
#     conv1 = tf.layers.conv2d(
#         inputs=input_layer,
#         filters=32,
#         kernel_size=[5, 5],# or kernel_size=5
#         padding="same", #same; output tensor should have the same width and height as input tensor; otherwise: valid(default)
#         activation=tf.nn.relu,
#         use_bias=True,
#         bias_initializer=tf.constant_initializer(0.0))
    
#     pool1 = tf.layers.max_pooling2d(
#         inputs=conv1,
#         pool_size=2,
#         strides=2)

#     norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,name='norm1')
#     # Convolutional Layer #2 and Pooling Layer #2
#     conv2 = tf.layers.conv2d(
#         inputs=pool1,
#         filters=64,
#         kernel_size=[5, 5],
#         padding="same",
#         activation=tf.nn.relu,
#         use_bias=True,
#         bias_initializer=tf.constant_initializer(0.0))
    
#     norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    
#     pool2 = tf.layers.max_pooling2d(
#         inputs=norm2,
#         pool_size=2,
#         strides=2)

#     # Dense Layer
#     pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
#     dense1 = tf.layers.dense(
#         inputs=pool2_flat,
#         units=1024, # number of neurons in the dense layer
#         activation=tf.nn.relu)
    
#     dropout1 = tf.layers.dropout(
#         inputs=dense1,
#         rate=0.4,
#         training= mode==tf.estimator.ModeKeys.TRAIN)
    
#     dense2 = tf.layers.dense(
#         inputs=dropout1,
#         units=1024, # number of neurons in the dense layer
#         activation=tf.nn.relu)
    
#     dropout2 = tf.layers.dropout(
#         inputs=dense2,
#         rate=0.4,
#         training= mode==tf.estimator.ModeKeys.TRAIN)

#     # Logits Layer
#     logits = tf.layers.dense(inputs=dropout2, units=10)
    logits = predicter(input_layer)
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

def unpickle(file):
        import pickle
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
            return dict # return dic keys: [b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data']
def main(unused_argv):
#     # Load training and eval data
#     mnist = tf.contrib.learn.datasets.load_dataset("mnist")
#     train_data = mnist.train.images # Returns np.array
#     train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
#     eval_data = mnist.test.images # Returns np.array
#     eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
#     mnist_classifier = tf.estimator.Estimator(
#     model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")


    # Data format:
    # data -- a 10000x3072 numpy array of uint8s. 
    #         Each row of the array stores a 32x32 colour image. 
    #         The first 1024 entries contain the red channel values, 
    #         the next 1024 the green, and the final 1024 the blue. 
    #         The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
    # labels -- a list of 10000 numbers in the range 0-99. The number at index i indicates the label of the ith image in the array data.

    # Load training data
    train_data = np.empty((0,3072))
    train_labels = np.empty((0,))
    for i in range(5):
        print('Getting data from batch', i+1)
        file = './Data/cifar-10/data_batch_'+str(i+1)
        train_set = unpickle(file)
        train_data = np.concatenate((train_data,train_set[b'data']), axis=0) # shape (50000, 3072) 50000 images of 32x32x3 values
        train_labels = np.concatenate((train_labels,np.asarray(train_set[b'labels'])), axis=0)
    
    print(train_data.shape)
    print(train_labels.shape)
    
    train_data = np.asarray(train_data, dtype=np.float32)
    train_labels = np.asarray(train_labels, dtype=np.int32)
    
    # Load testing data
    test_set = unpickle('./Data/cifar-10/test_batch')
    eval_data =np.asarray(test_set[b'data'], dtype=np.float32) # shape (10000, 3072) 50000 images of 32x32x3 values
    eval_labels = np.asarray(test_set[b'labels'], dtype=np.int32)
    #print(train_labels)
    mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./tmp/mnist_classifier")
    
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
          tensors=tensors_to_log, every_n_iter=50)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=BATCH_SIZE, # number of data in a minibatch
        num_epochs=256,
        shuffle=True) # shuffle training data
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

if __name__ == "__main__":
    tf.app.run()

Getting data from batch 1
Getting data from batch 2
Getting data from batch 3
Getting data from batch 4
Getting data from batch 5
(50000, 3072)
(50000,)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/mnist_classifier', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114abe278>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
dim (4096, 384)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./tmp/mnist_classifier/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.       